- ### After downloading the simulator, a few test drives with are done to get the hang of environment (Keyboard sensitivities, the road, etc.)
- ### The car was driven for a few rounds on the trail with an attempt to create a general dataset which includes moving backwards, swerving to sides, moving stable and steady, with and without acceleration.
- ### Logs and the corresponding jpg images can easily be downloaded

In [ ]:
import os, ntpath
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv("SelfDriving/log.csv", names = ["center", "left", "right", "steering", "throttle", "reverse", "speed"])
print(data.head())

center  \
0  D:\User-Majid\University_Courses\M.Sc\Courses\...   
1  D:\User-Majid\University_Courses\M.Sc\Courses\...   
2  D:\User-Majid\University_Courses\M.Sc\Courses\...   
3  D:\User-Majid\University_Courses\M.Sc\Courses\...   
4  D:\User-Majid\University_Courses\M.Sc\Courses\...   

                                                left  \
0  D:\User-Majid\University_Courses\M.Sc\Courses\...   
1  D:\User-Majid\University_Courses\M.Sc\Courses\...   
2  D:\User-Majid\University_Courses\M.Sc\Courses\...   
3  D:\User-Majid\University_Courses\M.Sc\Courses\...   
4  D:\User-Majid\University_Courses\M.Sc\Courses\...   

                                               right  steering  throttle  \
0  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
1  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
2  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
3  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
4  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   

   reverse     speed  
0      0.0  0.451284  
1      0.0  0.447647  
2      0.0  0.444939  
3      0.0  0.441358  
4      0.0  0.438693 

### Each data entry has columns:
1. center = jpg image directory for center angle viewpoint
2. left = jpg image directory for left angle viewpoint
3. right = jpg image directory for right angle viewpoint
4. steering = steering angle value (clockwise positive)
5. throttle = throttle value (acceleration)
6. reverse = boolean value representing whether moving backwards
7. speed = speed value

Only the central viewpoint images are worked with. To simplify stuff, direcory names are shortened

In [ ]:
data['center'] = data['center'].apply(lambda x: ntpath.basename(x))
print(data.head())
print(len(data['center']))

center  \
0  center_2019_02_18_14_33_39_945.jpg   
1  center_2019_02_18_14_33_40_017.jpg   
2  center_2019_02_18_14_33_40_090.jpg   
3  center_2019_02_18_14_33_40_160.jpg   
4  center_2019_02_18_14_33_40_231.jpg   

                                                left  \
0  D:\User-Majid\University_Courses\M.Sc\Courses\...   
1  D:\User-Majid\University_Courses\M.Sc\Courses\...   
2  D:\User-Majid\University_Courses\M.Sc\Courses\...   
3  D:\User-Majid\University_Courses\M.Sc\Courses\...   
4  D:\User-Majid\University_Courses\M.Sc\Courses\...   

                                               right  steering  throttle  \
0  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
1  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
2  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
3  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   
4  D:\User-Majid\University_Courses\M.Sc\Courses\...       0.0       0.0   

   reverse     speed  
0      0.0  0.451284  
1      0.0  0.447647  
2      0.0  0.444939  
3      0.0  0.441358  
4      0.0  0.438693

7458

### Take a look at data disribution

In [ ]:
hist, bins = np.histogram(data['steering'], 25)
plt.bar(bins[:-1], hist, width = 0.04)
centered_bins = (bins[:-1] + bins[1:]) / 2.0
plt.bar(centered_bins, hist, width = 0.04)
plt.xlabel("steering angle")
threshold = 300
plt.plot((-1, 1), (threshold, threshold))
plt.show()

<img src="./ThresholdShown.png">

#### Obviously there is way too many images with zero angle as a result of most of the road being straight. To remove this bias, images with steering value of 0 are curtailed to below the threshold of 300

In [ ]:
from sklearn.utils import shuffle
to_be_removed = []                                                              #list to collect indices of removable data
for i in range(25):
    lst = []                                                                      #list to collect indices
    for j in range(len(data['steering'])):
        if data['steering'][j] <= bins[i+1] and data['steering'][j] >= bins[i]:     #this means if the steering anlge falls in the bin
        lst.append(j)
    lst = shuffle(lst)                                                            #we need to shuffle the list in order to avoid truncating only the final portion of the road
    to_be_removed.extend(lst[threshold:])

data.drop(data.index[to_be_removed], inplace=True)
hist, b = np.histogram(data['steering'], 25)
centered_bins = (b[:-1] + b[1:]) / 2.0
plt.bar(centered_bins, hist, width = 0.04)
threshold = 300
plt.plot((-1, 1), (threshold, threshold))
plt.show()

<img src="ThresholdShown_curtailed.png">

### Creating train and test datasets

In [ ]:
imagepath = np.array(data['center'])
steer = np.array(data['steering'])
X_train, X_test, y_train, y_test = train_test_split(imagepath, steer, test_size = 0.2, random_state = 1)
imagepath.shape

(399,)

In [ ]:
s = X_train[0]
s = mpimg.imread(os.path.join('SelfDriving_simulation', 'IMG', s))
q = cv2.cvtColor(s, cv2.COLOR_RGB2YUV)                                     #reduce the noise and smooth out the image
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 10))
axs[0].imshow(s)
axs[1].imshow(q)
plt.show()

<img src="convertedcolors.png">